<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Load-the-Data" data-toc-modified-id="Load-the-Data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Load the Data</a></span><ul class="toc-item"><li><span><a href="#Download-data-(if-needed)" data-toc-modified-id="Download-data-(if-needed)-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Download data (if needed)</a></span></li><li><span><a href="#Read-in-log-file" data-toc-modified-id="Read-in-log-file-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Read in log file</a></span></li><li><span><a href="#Read-in-images-by-path-from-log-file" data-toc-modified-id="Read-in-images-by-path-from-log-file-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Read in images by path from log file</a></span></li></ul></li><li><span><a href="#Model" data-toc-modified-id="Model-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Model</a></span><ul class="toc-item"><li><span><a href="#Using-center-images-only" data-toc-modified-id="Using-center-images-only-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Using center images only</a></span><ul class="toc-item"><li><span><a href="#Evaluation" data-toc-modified-id="Evaluation-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Evaluation</a></span></li></ul></li></ul></li></ul></div>

In [ ]:
import pandas as pd
import numpy as np
import cv2

# Load the Data

We'll load the log data & also load the images (found in the log file).

## Download data (if needed)

In [ ]:
# Download data & unzip if it doesn't already exist
import os.path
from io import BytesIO
from urllib.request import urlopen
from zipfile import ZipFile

In [ ]:
def load_ext_file(data_zip_url, data_path='data/'):
    '''Download the zip file from URL and extract it to path (if specified).
    '''
    # Check if path already exits
    if not os.path.exists(data_path):
        with urlopen(data_zip_url) as zip_resp:
            with ZipFile(BytesIO(zip_resp.read())) as zfile:
                # Extract files into the data directory
                zfile.extractall(path=None)
       

In [ ]:
# Zip file contains the "data" and "__MACOSX" directories
load_ext_file(
    'https://d17h27t6h515a5.cloudfront.net/topher/2016/December/584f6edd_data/data.zip',
    data_path='data/'
)

## Read in log file

In [ ]:
def create_img_meas_dfs(log_csv, data_dir=None, orig_dir=None, skiprows=None):
    '''Creates DataFrames for the image paths and measurements using CSV path.
    
    Returns tuple of two DataFrames.
    '''
    data_header = [
        'image_center',
        'image_left',
        'image_right',
        'steer_angle', # [-1,1]
        'throttle', # boolen (if accelerating)
        'break', # boolean (if breaking)
        'speed' # mph
    ]

    df = pd.read_csv(
        log_csv,
        names=data_header,
        skiprows=skiprows
    )

    # Replace the original directory from dataset (if specified)
    if orig_dir and data_dir:
        for col in ['image_center','image_left','image_right']:
            df[col] = df[col].str.replace(orig_dir,data_dir)
    
    # Get specifics for each DF
    df_img_paths = df.iloc[:,:3]
    df_measurments = df.iloc[:,3:]
    
    return df_img_paths,df_measurments

In [ ]:
df_imgs, df_meas = create_img_meas_dfs(log_csv='data/driving_log.csv',skiprows=1)

display(df_imgs.head())

print('Stats for measurements:')
display(df_meas.describe())

## Read in images by path from log file

In [ ]:
center_images = []
left_images = []
right_images = []

# TODO: Fix since this is a slow and inefficient process
# Iterate over the different images
data_dir = 'data/'
for index,row in df_imgs.iterrows():
    center_images.append(cv2.imread(data_dir+row['image_center'].strip()))
    left_images.append(cv2.imread(data_dir+row['image_left'].strip()))
    right_images.append(cv2.imread(data_dir+row['image_right'].strip()))

In [ ]:
# Test to see if image reading works
import matplotlib.pyplot as plt

# sub
f = plt.figure(figsize=(25,25))
ax_left = f.add_subplot(1, 3, 1)
ax_center = f.add_subplot(1, 3, 2)
ax_right = f.add_subplot(1, 3, 3)

ax_center.imshow(center_images[0])
ax_left.imshow(left_images[0])
ax_right.imshow(right_images[0])

# Model

In [4]:
import tensorflow.keras as keras

## Using center images only

We'll try just using center images for training the model. If we simply put in the left and right images for the camera angle, we'd likely have issues with the model learning incorrect behavior. There are some techniques that could allow us to use these other images but for simplicity's sake we'll only use the center images for now.

In [5]:
# Creating a resuable default convolution
from functools import partial
DefaultConv2D = partial(keras.layers.Conv2D, kernel_initializer='he_normal',
                        kernel_size=3, activation='relu', padding='SAME')

In [ ]:
input_shape = center_images[0].shape

In [ ]:
# Based on https://developer.nvidia.com/blog/deep-learning-self-driving-cars/
model_list = [
    # Normalize the images
    keras.layers.Lambda(lambda x: (x/255.0) - 0.5, input_shape=input_shape),
    DefaultConv2D(filters=24, kernel_size=5, stride=2),
    keras.layers.MaxPooling2D(pool_size=2),
    DefaultConv2D(filters=36, kernel_size=5),
    keras.layers.MaxPooling2D(pool_size=2), 
    DefaultConv2D(filters=48),
    keras.layers.MaxPooling2D(pool_size=2),
    DefaultConv2D(filters=64),
    keras.layers.MaxPooling2D(pool_size=2),
    # Fully connected network
    keras.layers.Flatten(),
    keras.layers.Dense(units=1024, activation='relu'),
    keras.layers.Dropout(0.2),  # Dropout to regularize
    keras.layers.Dense(units=128, activation='relu'),
    keras.layers.Dropout(0.2),  # Dropout to regularize
    keras.layers.Dense(units=64, activation='relu'),
    keras.layers.Dropout(0.2),  # Dropout to regularize
    keras.layers.Dense(units=16, activation='relu'),
    keras.layers.Dense(units=1)
]

In [ ]:
# Adding in model to crop images first
model_list = (
    [model_list[0]] +
    # Crop out "unnessary images"
    [keras.layers.Cropping2D(cropping=((60,20), (0,0)))] +
    model_list[1:]
)

In [ ]:
model = keras.models.Sequential(model_list)

In [ ]:
model.compile(
    loss='mse', 
    optimizer='nadam'
)

In [ ]:
model.summary()

In [ ]:
# Only using center images for now
X = np.array(center_images)
y = df_meas['steer_angle'].values.reshape(-1,1)

In [ ]:
# Allow early stopping after not changing significantly
stop_after_5_no_change = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0.001,
    patience=5,
    restore_best_weights=True
)

history = model.fit(
    X,
    y, 
    epochs=30, 
    validation_split=0.2,
    callbacks=[stop_after_5_no_change]
)

### Evaluation

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline 

def eval_model(model, model_history, X, y, show=True):
    '''
    '''
    score = model.evaluate(X, y)
    print(f'Loss: {score:.2f}')

    if show:
        plt.plot(model_history.history['loss'], label='Loss (training data)')
        plt.plot(model_history.history['val_loss'], label='Loss (validation data)')
        plt.ylabel('Loss')
        plt.xlabel('Epoch')
        plt.legend(loc='upper right')
        plt.show()

Let's checkout how the previous model turned while training.

In [ ]:
eval_model(model, history, X, y)

In [ ]:
model.save('model.h5')